# Trying it out for fixed environment (or compromised environment)

In [1]:
%matplotlib inline

In [2]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.autograd import Variable

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from utils.ReplayMemory import * 
from utils.networks import *
from utils.agents import *
from utils.plotting import *
from utils.train_test_methods import *
from utils.params import *

from importlib import reload
import utils
reload(utils.ReplayMemory)
reload(utils.networks)
reload(utils.agents)
reload(utils.plotting)
reload(utils.train_test_methods)

<module 'utils.train_test_methods' from '/Users/zz/Documents/GT 20F/7000/GNN_experiments/utils/train_test_methods.py'>

In [4]:
input_type = U_VELOCITY
observe_type = O_VELOCITY
observe_action = O_ACTION
reward_mode=ALL_REWARD

## Try out Actor-Critic

In [ ]:
# Test out Actor-Critic methods. Note that tuning parameters might be required.
reload(utils.ReplayMemory)
reload(utils.networks)
reload(utils.agents)
reload(utils.plotting)
reload(utils.train_test_methods)
from utils.ReplayMemory import * 
from utils.networks import *
from utils.agents import *
from utils.plotting import *
from utils.train_test_methods import *

num_episode=500#500
test_interval=10#0
num_test=10#50
num_iteration=200
BATCH_SIZE=64#128
debug=False
num_sample=50
seed=22222

# rand_mode = NO_RAND
rand_mode = GAUSS_RAND

N_list = [5,10,20]
env_list = []
for N_ in N_list:
    env_list.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD, 
                 boundary_policy=SOFT_PENALTY, finish_reward_policy=REWARD_IF_CONSENSUS
        ).unwrapped
    )

AC2_list = []
for i,N_ in enumerate(N_list[:4]):
    AC2_list.append(
        AC2Agent(device, N_, env_list[i].nf, env_list[i].na, hidden, rand_modeA=rand_mode,
                 learning_rateA=0.01, learning_rateC=0.02)
    )

AC2_hist = []
AC2_loss = []
for i,N_ in enumerate(N_list[:4]):
    AC2_loss.append([])
    AC2_hist.append(
        train(AC2_list[i], env_list[i], 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
              update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC2_loss[i])
    )
    print("Finished training env with {0} agents for AC".format(N_))


In [ ]:
# additional testing
num_episode=1500
test_interval=5#0
num_test=10#50
num_iteration=200
N_list = [5,10,20]

for i,N_ in enumerate(N_list[:4]):
#     AC2_list[i].optimizerA.learning_rate = 0.05
#     AC2_list[i].optimizerC.learning_rate = 0.08
    AC2_hist[i] += train(AC2_list[i], env_list[i], 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
              update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC2_loss[i])
    print("Finished training env with {0} agents for AC".format(N_))

for i,N_ in enumerate(N_list[:4]):
    AC2_test_hist[i] += plot_test(AC2_list[i], env_list[i], fnames=['']*num_test,
            num_iteration=num_iteration, action_space=action_space, imdir='screencaps/',debug=debug)
    print("Finished testnig env with {0} agents for AC".format(N_))

In [ ]:
reload(utils.plotting)
from utils.plotting import *
skip = 1
plot_reward_hist([h[::skip] for h in AC2_hist], test_interval*skip, 
                 ['AC2_N{0}'.format(N_) for N_ in N_list[:4]], 
                 log=False, num_iteration=num_iteration, N_list=([1 for N in N_list[:4]]), bar=False)

In [ ]:
# Plot loss per # of agent
skip=1
plot_loss_hist(hists=[h[::skip] for h in AC2_loss], hist_names=['AC2_N{0}'.format(N_) for N_ in N_list[:4]], log=False, 
               num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION, bar=False)

In [51]:
### Save their animations and see how they behave.
for i,N_ in enumerate(N_list):
    plot_test(AC2_list[i], env_list[i], fnames=['AC2_dist_reward_test{0}_N{1}'.format(j,N_) for j in range(1)],
        num_iteration=100, action_space=action_space, imdir='screencaps/',debug=debug)

Finished episode  0  with  99 ;
cumulative reward =  -4838.5194699456115
Finished episode  0  with  99 ;
cumulative reward =  -20258.766523015714
Finished episode  0  with  99 ;
cumulative reward =  -28968.251461913635


<Figure size 432x288 with 0 Axes>

In [50]:
AC2_list[0].save_model("AC2_negGausstest_onlydistreward_normalized_N5")
AC2_list[1].save_model("AC2_negGausstest_onlydistreward_normalized_N10")
AC2_list[2].save_model("AC2_negGausstest_onlydistreward_normalized_N20")

Saving model to models/AC2Agent_AC2_negGausstest_onlydistreward_normalized_N5
Saving model to models/AC2Agent_AC2_negGausstest_onlydistreward_normalized_N10
Saving model to models/AC2Agent_AC2_negGausstest_onlydistreward_normalized_N20


#### The above, but uses all reward terms together

In [ ]:
num_episode=500
test_interval=10#0
num_test=10#50
env_list_allreward = []
N_list = [5,10,20]
for N_ in N_list:
    env_list_allreward.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=ALL_REWARD
        ).unwrapped
    )

AC2_list_allreward = []
for i,N_ in enumerate(N_list[:4]):
    AC2_list_allreward.append(
        AC2Agent(device, N_, env_list_allreward[i].nf, env_list_allreward[i].na, hidden, 
                 learning_rateA=0.01, learning_rateC=0.02)
    )

AC2_hist_allreward = []
AC2_loss_allreward = []
for i,N_ in enumerate(N_list[:4]):
    AC2_loss_allreward.append([])
    AC2_hist_allreward.append(
        train(AC2_list_allreward[i], env_list_allreward[i], 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
              update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC2_loss_allreward[i])
    )
    print("Finished training env with {0} agents for AC".format(N_))


In [ ]:
plot_reward_hist(AC2_hist_allreward, test_interval, 
                 ['AC2_N{0}'.format(N_) for N_ in N_list[:4]], 
                 log=False, num_iteration=num_iteration, N_list=([1 for N in N_list[:4]]), bar=False)

In [ ]:
# Plot loss
plot_loss_hist(hists=[h[5::skip] for h in AC2_loss_allreward], hist_names=['AC2_N{0}'.format(N_) for N_ in N_list[:4]], log=False, 
               num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION, bar=False)

In [48]:
AC2_list_allreward[0].save_model("AC2_allreward_normalized_N5")
AC2_list_allreward[1].save_model("AC2_allreward_normalized_N10")
AC2_list_allreward[2].save_model("AC2_allreward_normalized_N20")

Saving model to models/AC2Agent_AC2_allreward_normalized_N5
Saving model to models/AC2Agent_AC2_allreward_normalized_N10
Saving model to models/AC2Agent_AC2_allreward_normalized_N20


In [109]:
reload(utils.plotting)
from utils.plotting import *
### Save their animations and see how they behave.
for i,N_ in enumerate(N_list):
    plot_test(AC2_list[i], env_list[i], fnames=['AC2_dist_reward_test{0}_N{1}'.format(j,N_) for j in range(3)],
        num_iteration=100, action_space=action_space, imdir='screencaps/',debug=debug)
    plot_test(AC2_list_allreward[i], env_list_allreward[i], fnames=['AC2_all_reward_test{0}_N{1}'.format(j,N_) for j in range(3)],
        num_iteration=100, action_space=action_space, imdir='screencaps/',debug=debug)

Finished episode  0  with  99 ;
cumulative reward =  -4731.4494197514305
Finished episode  1  with  99 ;
cumulative reward =  -5153.693242743248
Finished episode  2  with  99 ;
cumulative reward =  -5081.763071406728
Finished episode  0  with  99 ;
cumulative reward =  -11308.227650814204
Finished episode  1  with  99 ;
cumulative reward =  -10975.637769948189
Finished episode  2  with  99 ;
cumulative reward =  -10856.320880929872
Finished episode  0  with  99 ;
cumulative reward =  -26163.48483412173
Finished episode  1  with  99 ;
cumulative reward =  -25082.043036814284
Finished episode  2  with  99 ;
cumulative reward =  -25481.96634182731


<Figure size 432x288 with 0 Axes>

In [110]:
# Clear recorded histories
# AC2_hist_allreward = []
# AC2_loss_allreward = []

### Use advantage terms

In [ ]:
reload(utils.ReplayMemory)
reload(utils.networks)
reload(utils.agents)
reload(utils.plotting)
reload(utils.train_test_methods)
from utils.ReplayMemory import * 
from utils.networks import *
from utils.agents import *
from utils.plotting import *
from utils.train_test_methods import *

num_episode=500#500
test_interval=10#0
num_test=10#50
num_iteration=200
BATCH_SIZE=128
debug=False
num_sample=50
seed=22222

N_list = [5,10,20]
env_list = []
for N_ in N_list:
    env_list.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD,
                 boundary_policy=SOFT_PENALTY, finish_reward_policy=REWARD_IF_CONSENSUS
        ).unwrapped
    )

AC3_list = []
for i,N_ in enumerate(N_list):
    AC3_list.append(
        AC3Agent(device, N_, env_list[i].nf, env_list[i].na, hidden, rand_modeA=rand_mode,
                 learning_rateA=0.01, learning_rateC=0.02)
    )

AC3_hist = []
AC3_loss = []
for i,N_ in enumerate(N_list):
    AC3_loss.append([])
    AC3_hist.append(
        train(AC3_list[i], env_list[i], 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
              update_mode=UPDATE_PER_EPISODE, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC3_loss[i])
    )
    print("Finished training env with {0} agents for AC".format(N_))

In [ ]:
# Keep training
num_episode=1000
test_interval=10
num_test=10
num_iteration=200

N_list = [5,10,20]
env_list = []
for N_ in N_list:
    env_list.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD
        ).unwrapped
    )

for i,N_ in enumerate(N_list):
#     AC3_list[i].optimizerA.learning_rate = 0.05
#     AC3_list[i].optimizerC.learning_rate = 0.08
    AC3_hist[i] += train(AC3_list[i], env_list[i], 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
              update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC3_loss[i])
    print("Finished training env with {0} agents for AC".format(N_))

for i,N_ in enumerate(N_list):
    AC3_test_hist[i] += plot_test(AC3_list[i], env_list[i], fnames=['']*num_test,
            num_iteration=num_iteration, action_space=action_space, imdir='screencaps/',debug=debug)
    print("Finished testnig env with {0} agents for AC".format(N_))

In [ ]:
skip=1
plot_reward_hist([h[::skip] for h in AC3_hist], test_interval*skip, 
                 ['AC3_N{0}'.format(N_) for N_ in N_list], 
                 log=False, num_iteration=num_iteration, N_list=([1 for N in N_list]), bar=True)

In [ ]:
# Plot loss per # of agent
skip=1
plot_loss_hist(hists=[h[::skip] for h in AC3_loss], hist_names=['AC3_N{0}'.format(N_) for N_ in N_list], log=False, 
               num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION, bar=False)

In [89]:
AC3_list[0].save_model("AC3_NegGauss_distreward_normalized_N5")
AC3_list[1].save_model("AC3_NegGauss_distreward_normalized_N10")
AC3_list[2].save_model("AC3_NegGauss_distreward_normalized_N20")

Saving model to models/AC3Agent_AC3_NegGauss_distreward_normalized_N5
Saving model to models/AC3Agent_AC3_NegGauss_distreward_normalized_N10


IndexError: list index out of range

In [88]:
### Save their animations and see how they behave.
for i,env_ in enumerate(env_list):
    plot_test(AC3_list[i], env_, fnames=['AC3_distreward_normalized_N{1}_test{0}'.format(j,env_.N) for j in range(1)],
        num_iteration=100, action_space=action_space, imdir='screencaps/',debug=debug)

Finished episode  0  with  99 ;
cumulative reward =  -5021.702440149778
Finished episode  0  with  99 ;
cumulative reward =  -12571.596065265274


<Figure size 432x288 with 0 Axes>

In [ ]:
#### The above, but with all reward terms

In [ ]:
num_episode=500
test_interval=10#0
num_test=10#50
env_list_allreward = []
N_list = [5,10,20]
for N_ in N_list:
    env_list_allreward.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=ALL_REWARD
        ).unwrapped
    )

AC3_list_allreward = []
for i,N_ in enumerate(N_list):
    AC3_list_allreward.append(
        AC2Agent(device, N_, env_list_allreward[i].nf, env_list_allreward[i].na, hidden, 
                 learning_rateA=0.01, learning_rateC=0.02)
    )

AC3_hist_allreward = []
AC3_loss_allreward = []
for i,N_ in enumerate(N_list):
    AC3_loss_allreward.append([])
    AC3_hist_allreward.append(
        train(AC3_list_allreward[i], env_list_allreward[i], 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
              update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC3_loss_allreward[i])
    )
    print("Finished training env with {0} agents for AC".format(N_))

In [ ]:
skip = 1
plot_reward_hist([h[2::skip] for h in AC3_hist_allreward], test_interval*skip, 
                 ['AC3_N{0}'.format(N_) for N_ in N_list], 
                 log=False, num_iteration=num_iteration, N_list=([1 for N in N_list]), bar=False)

In [ ]:
# Plot loss
skip=1
plot_loss_hist(hists=[h[5::skip] for h in AC3_loss_allreward], hist_names=['AC3_N{0}'.format(N_) for N_ in N_list], log=False, 
               num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION, bar=False)

In [49]:
AC3_list_allreward[0].save_model("AC3_allreward_normalized_N5")
AC3_list_allreward[1].save_model("AC3_allreward_normalized_N10")
AC3_list_allreward[2].save_model("AC3_allreward_normalized_N20")

Saving model to models/AC2Agent_AC3_allreward_normalized_N5
Saving model to models/AC2Agent_AC3_allreward_normalized_N10
Saving model to models/AC2Agent_AC3_allreward_normalized_N20


In [117]:
reload(utils.plotting)
from utils.plotting import *
### Save their animations and see how they behave.
for i,N_ in enumerate(N_list):
    plot_test(AC3_list[i], env_list[i], fnames=['AC3_dist_reward_test{0}_N{1}'.format(j,N_) for j in range(1)],
        num_iteration=100, action_space=action_space, imdir='screencaps/',debug=debug)

Finished episode  0  with  99 ;
cumulative reward =  -4878.524448529784
Finished episode  1  with  99 ;
cumulative reward =  -4878.9808283917355
Finished episode  2  with  99 ;
cumulative reward =  -4935.346769331861
Finished episode  0  with  99 ;
cumulative reward =  -38275.258413423464
Finished episode  1  with  99 ;
cumulative reward =  -40579.144749352716
Finished episode  2  with  99 ;
cumulative reward =  -45368.396375618795
Finished episode  0  with  99 ;
cumulative reward =  -124197.28412464196
Finished episode  1  with  99 ;
cumulative reward =  -159430.7937167494
Finished episode  2  with  99 ;
cumulative reward =  -119916.70460990266


<Figure size 432x288 with 0 Axes>

In [111]:
# Clear some histories
# AC3_hist_allreward = []
# AC3_loss_allreward = []

### Try out AC with more interesting reward options

In [ ]:
# Test out Actor-Critic methods. Note that tuning parameters might be required.
reload(utils.ReplayMemory)
reload(utils.networks)
reload(utils.agents)
reload(utils.plotting)
reload(utils.train_test_methods)
from utils.ReplayMemory import * 
from utils.networks import *
from utils.agents import *
from utils.plotting import *
from utils.train_test_methods import *

num_episode=500
test_interval=10#0
num_test=10#50
num_iteration=200
BATCH_SIZE=64#128
debug=False
num_sample=50
seed=22222

N_listv = [5,10]#,20]
env_listv = []
for N_ in N_listv:
    # Distance-based reward only, with hard penalty on touching the boundary.
    # Control group that doesn't give surviving reward, and instead stops immediately.
    env_listv.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD
        ).unwrapped
    )
    # Drop-dead immediately on touching the boundary
    env_listv.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD,
                 boundary_policy=DEAD_ON_TOUCH
        ).unwrapped
    )
    # Hard penalty on boundary, coupled with positive convergence reward. This means it never stops on consensus.
    env_listv.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD, 
                 finish_reward_policy=REWARD_IF_CONSENSUS
        ).unwrapped
    )
    # Soft penalty with consensus rewards. I don't expect it to successfully discover
    # achieving consensus would bring reward, though.
    env_listv.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD,
                 boundary_policy=SOFT_PENALTY, finish_reward_policy=REWARD_IF_CONSENSUS
        ).unwrapped
    )
    # Comparison group that uses dist and actuation rewards, with hard penalty
    env_listv.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, 
                 reward_mode=(DIST_REWARD|ACT_REWARD)
        ).unwrapped
    )

labels = ['hard_bound_zero_posReward', 
          'dead_bound_zero_posReward', 
          'hard_bound_cumu_posReward', 
          'soft_bound_cumu_posReward', 
          'hard_bound_zero_posReward_v_penalty']*2
labels = [labels[i]+'_N{0}'.format(env_.N) for i,env_ in enumerate(env_listv)]

AC2_listv = []
for i,env_ in enumerate(env_listv):
    AC2_listv.append(
        AC2Agent(device, env_.N, env_.nf, env_.na, hidden, rand_modeA=rand_mode,
                 learning_rateA=0.01, learning_rateC=0.02)
    )

AC2_histv = []
AC2_lossv = []
for i,env_ in enumerate(env_listv):
    AC2_lossv.append([])
    AC2_histv.append(
        train(AC2_listv[i], env_, 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
#               update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC2_loss[i])
              # Using normalization might ruine the effort?
              update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC2_lossv[i])
    )
    print("Finished training env with {0} agents for AC".format(env_.N))

In [ ]:
# additional training
num_episode=500
test_interval=10
num_test=10#50
num_iteration=200

for i,env_ in enumerate(env_listv):
    AC2_listv[i].optimizerA.learning_rate = 0.1
    AC2_listv[i].optimizerC.learning_rate = 0.1
    AC2_histv[i] += train(AC2_listv[i], env_, 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
              update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC2_lossv[i])
    print("Finished training "+labels[i])


In [ ]:
reload(utils.plotting)
from utils.plotting import *

skip = 1
i = 9
plot_reward_hist([AC2_histv[i][::skip]], test_interval*skip, 
                 [labels[i]], # ['AC2_N{0}'.format(env_list[i].N)], 
                 log=False, num_iteration=num_iteration, 
                 N_list=[env_listv[i].N], # ([1 for env_ in env_list]), 
                 bar=True)

In [ ]:
# Plot loss per # of agent
skip=1
# i = 0
# plot_loss_hist(hists=[AC2_loss[i][::skip]], 
#                hist_names=[labels[i]], 
#                log=False, num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION, bar=True)
plot_loss_hist(hists=[h[::skip] for h in AC2_lossv], hist_names=labels, 
               log=False, num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION, bar=False)

In [55]:
for i,lab in enumerate(labels):
    AC2_listv[i].save_model("AC2_GaussTest_distreward_normalized_"+lab)

Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_hard_bound_zero_posReward_N5
Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_dead_bound_zero_posReward_N5
Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_hard_bound_cumu_posReward_N5
Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_soft_bound_cumu_posReward_N5
Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_hard_bound_zero_posReward_v_penalty_N5
Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_hard_bound_zero_posReward_N10
Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_dead_bound_zero_posReward_N10
Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_hard_bound_cumu_posReward_N10
Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_soft_bound_cumu_posReward_N10
Saving model to models/AC2Agent_AC2_GaussTest_distreward_normalized_hard_bound_zero_posReward_v_penalty_N10


In [57]:
### Save their animations and see how they behave.
for i,env_ in enumerate(env_listv):
    plot_test(AC2_listv[i], env_, fnames=['AC2_distreward_normalized'+labels[i]+'_test{0}'.format(j) for j in range(1)],
        num_iteration=100, action_space=action_space, imdir='screencaps/',debug=debug)

Took  21  steps to converge
Finished episode  0  with  21 ;
cumulative reward =  -54302210.1283577
Took  6  steps to converge
Finished episode  0  with  6 ;
cumulative reward =  -10100299.503687605
Took  21  steps to converge
Finished episode  0  with  21 ;
cumulative reward =  -53702239.19595753
Finished episode  0  with  99 ;
cumulative reward =  -20397.49118748304
Took  20  steps to converge
Finished episode  0  with  20 ;
cumulative reward =  -53702669.53144223
Took  22  steps to converge
Finished episode  0  with  22 ;
cumulative reward =  -103706912.39399435
Took  15  steps to converge
Finished episode  0  with  15 ;
cumulative reward =  -10104119.727468826
Took  16  steps to converge
Finished episode  0  with  16 ;
cumulative reward =  -105802616.69716345
Finished episode  0  with  99 ;
cumulative reward =  -75435.10133687276
Took  26  steps to converge
Finished episode  0  with  26 ;
cumulative reward =  -109013444.81394063


<Figure size 432x288 with 0 Axes>

### Try using Additive noise instead of Gaussian or noise-free
The environment reward configs here are slightly different from the ones above

In [ ]:
# Test out Actor-Critic with noise
reload(utils.ReplayMemory)
reload(utils.networks)
reload(utils.agents)
reload(utils.plotting)
reload(utils.train_test_methods)
from utils.ReplayMemory import * 
from utils.networks import *
from utils.agents import *
from utils.plotting import *
from utils.train_test_methods import *

num_episode=500
test_interval=10
num_test=10#50
num_iteration=200

N_list = [5,10,20]
env_listr = []
for N_ in N_list:
    # Distance-based reward only, with hard penalty on touching the boundary.
    # Control group that doesn't give surviving reward, and instead stops immediately.
    env_listr.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD
        ).unwrapped
    )
    # Drop-dead immediately on touching the boundary
    env_listr.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD,
                 boundary_policy=DEAD_ON_TOUCH
        ).unwrapped
    )
    # Hard penalty on boundary, coupled with positive convergence reward. This means it never stops on consensus.
    env_listr.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD, 
                 finish_reward_policy=REWARD_IF_CONSENSUS
        ).unwrapped
    )
    # Soft penalty with consensus rewards. I don't expect it to successfully discover
    # achieving consensus would bring reward, though.
    env_listr.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, reward_mode=DIST_REWARD,
                 boundary_policy=SOFT_PENALTY, finish_reward_policy=REWARD_IF_CONSENSUS
        ).unwrapped
    )
    # Comparison group that uses dist and actuation rewards, with hard penalty
    env_listr.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION, 
                 reward_mode=(DIST_REWARD|ACT_REWARD)
        ).unwrapped
    )

AC2_listr = []
for i,env_ in enumerate(env_listr):
    AC2_listr.append(
        AC2Agent(device, env_.N, env_.nf, env_.na, hidden, add_noise=True,
                 learning_rateA=0.01, learning_rateC=0.02)
    )

AC2_histr = []
AC2_lossr = []
for i,env_ in enumerate(env_listr):
    AC2_lossr.append([])
    AC2_histr.append(
        train(AC2_listr[i], env_, 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
#               update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC2_loss[i])
              # Using normalization might ruine the effort?
              update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC2_lossr[i])
    )
    print("Finished training env with {0} agents for AC".format(env_.N))

In [ ]:
# additional training
num_episode=500
test_interval=10
num_test=10#50
num_iteration=200

for i,env_ in enumerate(env_listr):
    AC2_listr[i].optimizerA.learning_rate = 0.1
    AC2_listr[i].optimizerC.learning_rate = 0.1
    AC2_histr[i] += train(AC2_listr[i], env_, 
              num_episode=num_episode, test_interval=test_interval, num_test=num_test, num_iteration=num_iteration, 
              BATCH_SIZE=BATCH_SIZE, num_sample=num_sample, action_space=[-1,1], debug=debug,
              update_mode=UPDATE_PER_ITERATION, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=AC2_lossr[i])
    print("Finished training "+labels[i])


In [ ]:
skip = 1
i = 4
plot_reward_hist([AC2_histr[i][::skip]], test_interval*skip, 
                 [labels[i]], # ['AC2_N{0}'.format(env_list[i].N)], 
                 log=False, num_iteration=num_iteration, 
                 N_list=[env_listr[i].N], # ([1 for env_ in env_list]), 
                 bar=True)

In [ ]:
# Plot loss per # of agent
skip=1
# i = 0
# plot_loss_hist(hists=[AC2_lossr[i][::skip]], 
#                hist_names=[labels[i]], 
#                log=False, num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION, bar=True)
plot_loss_hist(hists=[h[::skip] for h in AC2_lossr], hist_names=labels, 
               log=False, num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION, bar=False)

In [ ]:
for i,lab in enumerate(labels[:5]):
    AC2_listr[i].save_model("AC2_randnoise_distreward_normalized_"+lab)

In [ ]:
### Save their animations and see how they behave.
for i,env_ in enumerate(env_listr):
    plot_test(AC2_listr[i], env_, fnames=['AC2_noise_distreward_normalized'+labels[i]+'_test{0}'.format(j) for j in range(2)],
        num_iteration=100, action_space=action_space, imdir='screencaps/',debug=debug)

### DDPG

In [ ]:
reload(utils.agents)
from utils.agents import *

num_episode=500
test_interval=50
num_test=50
num_iteration=200
BATCH_SIZE=128
debug=False
num_sample=50

# Do DDPG
DDPG_list = []
for i,N_ in enumerate(N_list):
    DDPG_list.append(
        DDPGAgent(device, N_, env_list[i].no, env_list[i].na, hidden)
    )
DDPG_list.append(
    DDPGAgent(device, 10, env.no, env.na, hidden)
)

DDPG_hist = []
for i,N_ in enumerate(N_list):
    DDPG_hist.append(
        train(DDPG_list[i], env_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug)
    )
    print("Finished training env with {0} agents for AC".format(N_))
DDPG_hist.append(
    train(DDPG_list[-1], env, 
          num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug)
)
    
DDPG_test_hist = []
for i,N_ in enumerate(N_list):
    DDPG_test_hist.append(
        plot_test(DDPG_list[i], env_list[i], fnames=['']*num_test,
            num_iteration=num_iteration, action_space=action_space, imdir='screencaps/', debug=debug)
    )
    print("Finished testnig env with {0} agents for AC".format(N_))
DDPG_test_hist.append(
    plot_test(DDPG_list[-1], env, fnames=['']*num_test,
        num_iteration=num_iteration, action_space=action_space, imdir='screencaps/', debug=debug)
)

In [ ]:

DDPG_test_hist = []
for i,N_ in enumerate(N_list):
    DDPG_test_hist.append(
        plot_test(DDPG_list[i], env_list[i], fnames=['']*num_test,
            num_iteration=num_iteration, action_space=action_space, imdir='screencaps/', debug=debug)
    )
    print("Finished testnig env with {0} agents for AC".format(N_))
DDPG_test_hist.append(
    plot_test(DDPG_list[-1], env, fnames=['']*num_test,
        num_iteration=num_iteration, action_space=action_space, imdir='screencaps/', debug=debug)
)

In [ ]:
plot_reward_hist(DDPG_hist, test_interval, 
                 ['DDPG_N{0}'.format(N_) for N_ in N_list]+['DDPG_N10'], 
                 log=False, num_iteration=num_iteration, N_list=([np.sqrt(N) for N in N_list]+[np.sqrt(10)])*2)

In [ ]:
plot_reward_hist([[l] for l in DDPG_test_hist], test_interval, 
                 ['DDPG_N{0}'.format(N_) for N_ in N_list]+['DDPG_N10'], 
                 log=False, num_iteration=num_iteration, N_list=([N for N in N_list]+[10])*2)

In [ ]:

def save_model(agent, suffix="", agent_path=None):
    if not os.path.exists('models/'):
        os.makedirs('models/')

    if len(suffix) <= 0:
        suffix = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    if agent_path is None:
        agent_path = "models/{}_{}".format(agent.name, suffix)
    print('Saving model to {}'.format(agent_path))
    torch.save(agent.netA.state_dict(), agent_path+"_A")
    torch.save(agent.netC.state_dict(), agent_path+"_C")
    
for i,N_ in enumerate(N_list+[10]):
    save_model(AC2_list[i],'AC2_test2_N{0}'.format(N_))
    
def save_model(agent, suffix="", agent_path=None):
    if not os.path.exists('models/'):
        os.makedirs('models/')

    if len(suffix) <= 0:
        suffix = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    if agent_path is None:
        agent_path = "models/{}_{}".format(agent.name, suffix)
    print('Saving model to {}'.format(agent_path))
    torch.save(agent.netA.state_dict(), agent_path+"_A")
    torch.save(agent.netC.state_dict(), agent_path+"_C")
    torch.save(agent.netAT.state_dict(), agent_path+"_AT")
    torch.save(agent.netCT.state_dict(), agent_path+"_CT")
    
for i,N_ in enumerate(N_list+[10]):
    save_model(DDPG_list[i],'DDPG_test2_N{0}'.format(N_))

### Test out different training setups and compare
using the new reward

In [ ]:
reload(utils.networks)
reload(utils.agents)
reload(utils.train_test_methods)
from utils.networks import *
from utils.agents import *
from utils.train_test_methods import *

num_episode=250
test_interval=25
num_test=25
num_iteration=200
BATCH_SIZE=128
debug=False
num_sample=50
transfer_num_episode=200
seed=2333333

# Bulid environments with different number of agents
env_list = []
env5_list = []
N_list = [5]#, 10, 20, 32]#, 50, 64]
for N_ in N_list:
    env_list.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_NO_ACTION
        ).unwrapped
    )
    env5_list.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION
        ).unwrapped
    )

# Initialize 4 different groups of agents
# Type 1: Learn from scratch using cumulative reward 
RA_cumu_list = []
# Type 2: Learn from scratch using non-cumulative reward 
RA_inst_list = []
# Type 3: Learn from scratch using normalized cumulative reward 
RA_norm_list = []
# Type 4: Transfer learning to the new environment from models saved using old reward function
RA_trsf_list = []
RApath = 'models/RewardAgent_RA32_master_agent'
prevN = 32
# Type 4, v2: Trans train using cumulative reward
RA_trfc_list = []
# Type 5: Learn while observing others' past actions
RA_obac_list = []

for i,N_ in enumerate(N_list):
    RA_cumu_list.append(
        RewardAgent(device, N_, env_list[i].no, env_list[i].na, hidden, learning_rate=0.1)
    )
    RA_inst_list.append(
        RewardAgent(device, N_, env_list[i].no, env_list[i].na, hidden, learning_rate=0.1)
    )
    RA_norm_list.append(
        RewardAgent(device, N_, env_list[i].no, env_list[i].na, hidden, learning_rate=0.1)
    )
    RA_trsf_list.append(
        RewardAgent(device, N_, env_list[i].no, env_list[i].na, hidden, learning_rate=0.1, prevN=prevN, load_path=RApath)
    )
    RA_trfc_list.append(
        RewardAgent(device, N_, env_list[i].no, env_list[i].na, hidden, learning_rate=0.1, prevN=prevN, load_path=RApath)
    )
    RA_obac_list.append(
        RewardAgent(device, N_, env5_list[i].nf, env_list[i].na, hidden, learning_rate=0.1)
    )

In [ ]:
# Train those agents one by one, I guess
RA_cumu_hists = []
RA_inst_hists = []
RA_norm_hists = []
RA_trsf_hists = []
RA_obac_hists = []
RA_cumu_lossess = [[] for i in N_list]
RA_inst_lossess = [[] for i in N_list]
RA_norm_lossess = [[] for i in N_list]
RA_trsf_lossess = [[] for i in N_list]
RA_obac_lossess = [[] for i in N_list]
RA_trfc_lossess = [[] for i in N_list]
RA_cumu_lr_hist = [[] for i in N_list]
RA_inst_lr_hist = [[] for i in N_list]
RA_norm_lr_hist = [[] for i in N_list]
RA_trsf_lr_hist = [[] for i in N_list]
RA_obac_lr_hist = [[] for i in N_list]
RA_trfc_lr_hist = [[] for i in N_list]
for i,N_ in enumerate(N_list):
    RA_cumu_hists.append(
        train(RA_cumu_list[i], env_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_YES, loss_history=RA_cumu_lossess[i], lr_history=RA_cumu_lr_hist[i]
        )
    )
    RA_inst_hists.append(
        train(RA_inst_list[i], env_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_NO, loss_history=RA_inst_lossess[i], lr_history=RA_inst_lr_hist[i]
        )
    )
    RA_norm_hists.append(
        train(RA_norm_list[i], env_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=RA_norm_lossess[i], lr_history=RA_norm_lr_hist[i]
        )
    )
    RA_trsf_hists.append(
        train(RA_trsf_list[i], env_list[i], 
              transfer_num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_NO, loss_history=RA_trsf_lossess[i], lr_history=RA_trsf_lr_hist[i]
        )
    )
    RA_obac_hists.append(
        train(RA_obac_list[i], env5_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_YES, loss_history=RA_obac_lossess[i], lr_history=RA_obac_lr_hist[i]
        )
    )
    RA_trfc_hists.append(
        train(RA_trfc_list[i], env_list[i], 
              transfer_num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_YES, loss_history=RA_trfc_lossess[i], lr_history=RA_trfc_lr_hist[i]
        )
    )
    print("Finished training env with {0} agents for RA".format(N_))

In [ ]:
num_episode=250
for i,N_ in enumerate(N_list):
    RA_cumu_hists[i]+=train(RA_cumu_list[i], env_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_YES, loss_history=RA_cumu_lossess[i]
        )
    
    RA_inst_hists[i]+=train(RA_inst_list[i], env_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_NO, loss_history=RA_inst_lossess[i]
        )
    
    RA_norm_hists[i]+=train(RA_norm_list[i], env_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_YES_NORMALIZE, loss_history=RA_norm_lossess[i]
        )
    

In [ ]:
RA_trfc_hists = []
for i,N_ in enumerate(N_list):
    RA_trfc_hists.append(
        train(RA_trfc_list[i], env_list[i], 
              transfer_num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
              seed=seed, reward_mode=FUTURE_REWARD_YES
        )
    )

In [ ]:
for i,N_ in enumerate(N_list):
    RA_cumu_list[i].save_model('rewardV2_N{0}_cumu_test1'.format(N_))
    RA_inst_list[i].save_model('rewardV2_N{0}_inst_test1'.format(N_))
    RA_norm_list[i].save_model('rewardV2_N{0}_norm_test1'.format(N_))
    RA_trsf_list[i].save_model('rewardV2_N{0}_trsf_test1'.format(N_))
    RA_obac_list[i].save_model('rewardV2_N{0}_obac_test1'.format(N_))
    RA_trfc_list[i].save_model('rewardV2_N{0}_trfc_test1'.format(N_))

In [ ]:
# Plot per group
comb_hists = [RA_cumu_hists,RA_inst_hists,RA_norm_hists,RA_trsf_hists,RA_trfc_hists,RA_obac_hists]
comb_names = [['RA_{1}_N{0}'.format(N_,name) for N_ in N_list] for name in [
    'cum_reward','no_cum_reward','cum_norm_reward','tf_no_cum','tf_cum_reward','observe_action_cum']]
i = 5
plot_reward_hist(comb_hists[i], test_interval, 
                 comb_names[i], 
                 log=False, num_iteration=num_iteration, N_list=[1 for N_ in N_list])

In [ ]:
# Plot per # of agent
cumb_hists = [[RA_cumu_hists[i],RA_inst_hists[i],RA_norm_hists[i],
               RA_trsf_hists[i],RA_trfc_hists[i],RA_obac_hists[i]] for i in range(len(N_list))]
cumb_names = [['RA_{1}_N{0}'.format(N_,name) for name in [
    'cum_reward','no_cum_reward','cum_norm_reward','tf_no_cum','tf_cum_reward','observe_action_cum']] for N_ in N_list]
j = 3
plot_reward_hist(cumb_hists[j], test_interval, 
                 cumb_names[j], 
                 log=False, num_iteration=num_iteration, N_list=[N_list[j]]*6)

In [ ]:
reload(utils.plotting)
from utils.plotting import *
# Plot loss per # of agent
cumb_hists = [[RA_cumu_lossess[i],RA_inst_lossess[i],RA_norm_lossess[i],
               RA_trsf_lossess[i],RA_trfc_lossess[i],RA_obac_lossess[i]] for i in range(len(N_list))]
cumb_names = [['RA_{1}_N{0}'.format(N_,name) for name in [
    'cum_reward','no_cum_reward','cum_norm_reward','tf_no_cum','tf_cum_reward','observe_action_cum']] for N_ in N_list]

j = 0
plot_loss_hist(hists=cumb_hists[j], hist_names=cumb_names[j], log=False, 
               num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION)

### How about using CNN?

In [ ]:
reload(utils.networks)
reload(utils.agents)
reload(utils.train_test_methods)
from utils.networks import *
from utils.agents import *
from utils.train_test_methods import *

num_episode=500
test_interval=25
num_test=25
num_iteration=200
BATCH_SIZE=256
debug=False
num_sample=50
seed=2333333
hidden=8

NCN_list = [5,10] 
envCNN_list = []
for N_ in NCN_list:
    envCNN_list.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION
        ).unwrapped
    )
LA_CNN_list = []
RA_CNN_list = []
for i,N_ in enumerate(NCN_list):
    LA_CNN_list.append(
        LearnerCNNAgent(device, N_, ns=envCNN_list[i].nf, na=env_list[i].na, hidden=hidden, n_hid=2, in_features=1)
    )
    RA_CNN_list.append(
        RewardCNNAgent(device, N_, ns=envCNN_list[i].nf, na=env_list[i].na, hidden=hidden, n_hid=2, in_features=1)
    )
LA_CNN_hists = []
RA_CNN_hists = []
for i,N_ in enumerate(NCN_list):
    LA_CNN_hists.append(
        train(LA_CNN_list[i], envCNN_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
                             seed=seed, reward_mode=FUTURE_REWARD_YES, update_mode=UPDATE_PER_EPISODE)
    )
    RA_CNN_hists.append(
        train(RA_CNN_list[i], envCNN_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
                             seed=seed, reward_mode=FUTURE_REWARD_YES, update_mode=UPDATE_PER_EPISODE)
    )
    print("Finished training env with {0} agents for CNN".format(N_))

In [ ]:
# Keep training those lazy gradients
num_episode=2600
test_interval=20
num_test=25
num_iteration=200
BATCH_SIZE=256
debug=False
num_sample=50
seed=2333333
hidden=8

for i,N_ in enumerate(NCN_list):
    LA_CNN_hists[i] += train(LA_CNN_list[i], envCNN_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
                             seed=seed, reward_mode=FUTURE_REWARD_YES)
    RA_CNN_hists[i] += train(RA_CNN_list[i], envCNN_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
                             seed=seed, reward_mode=FUTURE_REWARD_YES)
    print("Finished training env with {0} agents for CNN".format(N_))

In [ ]:
# Plot results
plot_reward_hist(LA_CNN_hists, test_interval, ['LA_CNN_N{0}'.format(N_) for N_ in NCN_list], 
                 log=False, num_iteration=num_iteration, N_list=[1 for N_ in NCN_list])

In [ ]:
plot_reward_hist(RA_CNN_hists, test_interval, ['RA_CNN_N{0}'.format(N_) for N_ in NCN_list], 
                 log=False, num_iteration=num_iteration, N_list=[1 for N_ in NCN_list])

In [ ]:
LA_CNN_list[0].save_model("LA_CNN_test1_3000epi_N5")
LA_CNN_list[1].save_model("LA_CNN_test1_3000epi_N10")
RA_CNN_list[0].save_model("RA_CNN_test1_3000epi_N5")
RA_CNN_list[1].save_model("LA_CNN_test1_3000epi_N10")

In [ ]:
reload(utils.networks)
reload(utils.agents)
reload(utils.train_test_methods)
from utils.networks import *
from utils.agents import *
from utils.train_test_methods import *

# Additional training and testing and probing
num_episode=1#200
test_interval=20
num_test=1#25
num_iteration=200
BATCH_SIZE=32
debug=False
num_sample=50
seed=2333333
hidden=8

i = 0
N_ = 5
LCtest = LearnerCNNAgent(device, N_, ns=envCNN_list[i].nf, na=env_list[i].na, hidden=hidden, n_hid=2, in_features=1)
RCtest = RewardCNNAgent(device, N_, ns=envCNN_list[i].nf, na=env_list[i].na, hidden=hidden, n_hid=2, in_features=1)

LCtesth = train(LCtest, envCNN_list[i], 
          num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug)
RCtesth = train(RCtest, envCNN_list[i], 
          num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug)
print("Finished training env with {0} agents for CNN".format(N_))

### How about trying ResNet or similar structure?

In [ ]:
reload(utils.networks)
reload(utils.agents)
reload(utils.train_test_methods)
from utils.networks import *
from utils.agents import *
from utils.train_test_methods import *

num_episode=50
test_interval=5
num_test=10#25
num_iteration=50#200
BATCH_SIZE=64
debug=False
num_sample=50
seed=2333333

NCN_list = [5] 
envCNN_list = []
for N_ in NCN_list:
    envCNN_list.append(
        gym.make('ConsensusEnv:ConsensusContEnv-v0', N=N_, dt=0.1, Delta=0.05,
              input_type=input_type, observe_type=observe_type, observe_action=O_ACTION
        ).unwrapped
    )
RA_RN_list = []
for i,N_ in enumerate(NCN_list):
    RA_RN_list.append(
        RewardRNAgent(device, N_, ns=envCNN_list[i].nf, na=env_list[i].na, n_hid=2, in_features=1)
    )
RA_RN_hists = []
RA_RN_lossess=[]
for i,N_ in enumerate(NCN_list):
    RA_RN_hists.append(
        train(RA_RN_list[i], envCNN_list[i], 
              num_episode, test_interval, num_test, num_iteration, BATCH_SIZE, num_sample, [-1,1], debug, 
                             seed=seed, reward_mode=FUTURE_REWARD_YES, update_mode=UPDATE_PER_EPISODE,
                             loss_history=RA_RN_lossess)
    )
    print("Finished training env with {0} agents for RN".format(N_))

In [ ]:
# Plot per # of agent
plot_reward_hist(RA_RN_hists, test_interval, ['RA_ResNet_N5'], 
                 log=False, num_iteration=num_iteration, N_list=NCN_list)

In [ ]:
reload(utils.plotting)
from utils.plotting import *
# Plot loss per # of agent
plot_loss_hist(hists=[RA_RN_lossess], hist_names=['RA_ResNet_N5'], log=False, 
               num_iteration=num_iteration, update_mode=UPDATE_PER_ITERATION)

### Play around with the gradient agent